In [1]:
import ray 
ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2022-02-10_16-16-11_691462_8844/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-02-10_16-16-11_691462_8844/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-02-10_16-16-11_691462_8844',
 'metrics_export_port': 61154,
 'gcs_address': '127.0.0.1:60297',
 'node_id': '5c92fdaa11f564da880dcd24dc3466090b67df5c4e696446c53873cc'}

In [2]:
from ray import tune
from ray.rllib.agents.dqn.dqn_torch_model import DQNTorchModel
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
import gym 
from typing import Sequence
from ray.rllib.utils.typing import ModelConfigDict
import torch 
import torch.nn as nn 

# custom model is processed before the fully connected layer 

class CustomModel(TorchModelV2, nn.Module):
    def __init__(
        self,
        obs_space: gym.spaces.Space,
        action_space: gym.spaces.Space,
        num_outputs: int,
        model_config: ModelConfigDict,
        name: str,

        ):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)
        self.linear1 = nn.Linear(self.obs_space.shape[0], 30)
        self.linear2 = nn.Linear(30,50)
        self.linear3 = nn.Linear(50, model_config['fcnet_hiddens'][0])
            
    def forward(self, input_dict, state, seq_lens):
        x =input_dict['obs']
        x = self.linear1(x)
        x = nn.functional.relu(x)
        x = self.linear2(x)
        x = nn.functional.relu(x)   
        x = self.linear3(x)
        return x, state

In [7]:
from ray.rllib.agents.dqn import DQNTrainer

tune.run("DQN", 
         config={"env":"CartPole-v0", 
                 "framework":"torch", 
                 "model":{"custom_model": CustomModel, 
                          "fcnet_hiddens":[128],
                          "fcnet_activation":"relu"}
                 }, 
         stop={"training_iteration":5} , 
         checkpoint_freq=2,
         verbose=1)

2022-02-10 16:18:25,234	INFO tune.py:636 -- Total run time: 12.69 seconds (12.21 seconds for the tuning loop).
